In [1]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`


In this tutorial, we look at the internals of constraints creations.
Consider the `@constraint` call `@constraint(model, x == 1 - y)`, we can see below the code executed that is generated by this macro.

In [2]:
using JuMP
model = Model()
@variable(model, x)
@variable(model, y)
@macroexpand @constraint(model, x == 1 - y)

quote
    #= /home/blegat/.julia/packages/JuMP/jnmGG/src/macros.jl:278 =#
    (JuMP._valid_model)(model, :model)
    #= /home/blegat/.julia/packages/JuMP/jnmGG/src/macros.jl:279 =#
    begin
        #= /home/blegat/.julia/packages/JuMP/jnmGG/src/macros.jl:294 =#
        let
            #= /home/blegat/.julia/packages/JuMP/jnmGG/src/macros.jl:300 =#
            begin
                #= /home/blegat/.julia/packages/JuMP/jnmGG/src/macros.jl:620 =#
                begin
                    #5#q = (JuMP.Val){false}()
                    #= /home/blegat/.julia/packages/JuMP/jnmGG/src/macros.jl:410 =#
                    begin
                        #6###368 = (JuMP._destructive_add_with_reorder!)(#5#q, x)
                        begin
                            #7###369 = (JuMP._destructive_add_with_reorder!)(#6###368, -1.0, 1)
                            #8###367 = (JuMP._destructive_add_with_reorder!)(#7###369, -1.0, -1.0, y)
                        end
                    end
          

Let's break it down.


## Creating the function

We first evaluate the function, rewritting the expression to avoid the allocation intermediate expressions.

The type of the function change from `Val{false}` to `VariableRef` and then to `AffExpr` hence the need to use different variables to store the result.

In [3]:
func_0 = Val(false)
func_1 = JuMP._destructive_add_with_reorder!(func_0, x)

x

In [4]:
typeof(func_1)

VariableRef

In [5]:
func_2 = JuMP._destructive_add_with_reorder!(func_1, -1.0, 1)

x - 1

In [6]:
typeof(func_2)

GenericAffExpr{Float64,VariableRef}

In [7]:
func = JuMP._destructive_add_with_reorder!(func_2, -1.0, -1.0, y)
func

x + y - 1

## Creating the constraint

As the whole expression was move to the left-hand side, the right-hand side is zero.
The constraint can therefore be represented by `func`-in-`MOI.EqualTo(0.0)`.
When called from a macro, the first argument of `build_constraint` is a custom `error` function that prefix the error message with additional info useful for the user to relate the error with the macro written.

In [8]:
constraint = build_constraint(error, func, MOI.EqualTo(0.0))

ScalarConstraint{GenericAffExpr{Float64,VariableRef},MathOptInterface.EqualTo{Float64}}(x + y, MathOptInterface.EqualTo{Float64}(1.0))

Note that the constant (1.0) is moved to the set. This convention was chosen for scalar constraints since the constant cannot be stored in the function for the `Interval` set (i.e. we cannot clear the constants of both sides of the interval if they are different).

In [9]:
constraint_string(IJuliaMode, constraint)

"x + y = 1.0"

## Adding the constraint

No information was added to the model yet (note that the `model` is not passed as argument of the `build_constraint` function).
The `add_constraint` function adds the function to the underlying MOI backend.

In [10]:
cref = add_constraint(model, constraint)

x + y = 1.0

The constraint is now only stored in the MOI backend, the `constraint` object is discarded. However, it can be recovered from the MOI backend using `constraint_object`.

In [11]:
constraint_object(cref)

ScalarConstraint{GenericAffExpr{Float64,VariableRef},MathOptInterface.EqualTo{Float64}}(x + y, MathOptInterface.EqualTo{Float64}(1.0))

To illustrate the fact that no constraint data is stored in the JuMP model, we show below that if we add a constraint directly to the MOI backend with `MOI.add_constraint`, the JuMP model, won't see any difference from a constraint that was added using `JuMP.add_constraint`.

In [12]:
moi_x = MOI.SingleVariable(index(x))
moi_y = MOI.SingleVariable(index(y))
moi_func = 0.5 * moi_x - moi_y
ci = MOI.add_constraint(backend(model), moi_func, MOI.GreaterThan(2.0))

MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.GreaterThan{Float64}}(2)

As we can see below, JuMP prints the constraint as if it was added using `JuMP.add_constraint`.

In [13]:
model

A JuMP Model
Feasibility problem with:
Variables: 2
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.EqualTo{Float64}`: 1 constraint
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.GreaterThan{Float64}`: 1 constraint
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.
Names registered in the model: x, y

We can even recover the constraint object that would have been passed to `JuMP.add_constraint`.

In [14]:
constraint_object(JuMP.ConstraintRef(model, ci, ScalarShape()))

ScalarConstraint{GenericAffExpr{Float64,VariableRef},MathOptInterface.GreaterThan{Float64}}(0.5 x - y, MathOptInterface.GreaterThan{Float64}(2.0))